# Cloud evaluation -- Register custom evaluators

## Documentation

## Setup

### Environment setup
python 00_setup.py

### Common packages

In [ ]:
import os
import dotenv
from pathlib import Path

### Global settings

In [ ]:
# Global variables
PRIVATE = False
DATA_DIR = Path("data")
TMP_DIR = Path("tmp")

### Load environment variables

In [ ]:
# Import override environment variables from .env file
# or from private.env file if PRIVATE is True
dotenv.load_dotenv('.env' if not PRIVATE else 'private.env', override=True)

### Azure credentials

In [ ]:
# https://learn.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()

### Connect to Azure AI Foundry project

In [ ]:
from azure.ai.ml import MLClient

# Define ml_client to register custom evaluator
# https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.mlclient?view=azure-python
ml_client = MLClient(
       subscription_id=os.environ["AZURE_SUBSCRIPTION_ID"],
       resource_group_name=os.environ["AZURE_RESOURCE_GROUP_AI"],
       workspace_name=os.environ["AZURE_AI_FOUNDRY_PROJECT_NAME"],
       credential=credential
)

## Register custom evaluators

### Helper function

In [ ]:
import shutil
from promptflow.client import PFClient
from azure.ai.ml.entities import Model

def register_evaluator_from_class_helper(_class, _name:str, _description:str):
    
    # Cleanup previous artifacts
    _local_path = TMP_DIR / f"{_class.__name__}_pf"
    shutil.rmtree(_local_path, ignore_errors=True)    

    # Create a promptflow from the custom evaluator
    # https://microsoft.github.io/promptflow/reference/python-library-reference/promptflow-azure/promptflow.azure.html#promptflow.azure.PFClient
    pf_client = PFClient()
    pf_client.flows.save(entry=_class, path=_local_path) 

    # Register the evaluator to the Evaluator library
    # https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.model?view=azure-python
    _custom_evaluator = Model(
        path=_local_path,
        name=_name,
        description=_description
    )

    _registered_evaluator = ml_client.evaluators.create_or_update(_custom_evaluator)

    print("Registered evaluator id:", _registered_evaluator.id)
    print("Registered evaluator version:", _registered_evaluator.version)

### Register to AI Foundry evaluator library

In [ ]:
from custom.friendliness.friend import FriendlinessEvaluator
register_evaluator_from_class_helper(FriendlinessEvaluator, "FriendlinessEvaluator", "Prompt-based evaluator measuring response friendliness.")

In [ ]:
from custom.answer_len.answer_length import AnswerLengthEvaluator
register_evaluator_from_class_helper(AnswerLengthEvaluator, "AnswerLenEvaluator", "Evaluator calculating answer length.")

## Lookup registered evaluators in library

In [ ]:
_library_url = f"https://ai.azure.com/build/evaluation/evaluator?wsid=/subscriptions/{os.environ.get('AZURE_SUBSCRIPTION_ID')}/resourceGroups/{os.environ.get('AZURE_RESOURCE_GROUP_AI')}/providers/Microsoft.MachineLearningServices/workspaces/{os.environ.get('AZURE_AI_FOUNDRY_PROJECT_NAME')}"
print('Check out evaluator library: ', _library_url)